## Library Imports

In [1]:
from time import time
notebook_start_time = time()

In [2]:
import os
import re
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as DL
from torch.nn.utils import weight_norm as WN
from torchvision import models, transforms

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

## Constants and Utilities

In [3]:
SEED = 0
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PATH = "../input/petfinder-pawpularity-score"
PRETRAINED_WEIGHTS_PATH = "../input/pretrained-model-weights-all"
TRAINED_ANN_WEIGHTS_PATH = "../input/petfinder-d161-train"

MODEL_NAME = "densenet161"
FEATURE_LENGTH = 2208
NUM_FOLDS = len([name for name in os.listdir(TRAINED_ANN_WEIGHTS_PATH) if name[-2:] == "pt"])

IMAGE_SIZE = 224
TRANSFORM = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize([0.485, 0.456, 0.406],
                                                     [0.229, 0.224, 0.225]),
                                ])

sc_y = StandardScaler()

In [4]:
def breaker(num=50, char="*") -> None:
    print("\n" + num*char + "\n")


def head(x, no_of_ele=5) -> None:
    print(x[:no_of_ele])


def get_filenames_and_targets(path: str) -> tuple:
    df = pd.read_csv(os.path.join(path, "train.csv"), engine="python")
    filenames = df.iloc[:, 0].copy().values
    targets  = df.iloc[:, -1].copy().values
    return filenames, targets


def get_filenames(path: str) -> np.ndarray:
    df = pd.read_csv(os.path.join(path, "test.csv"), engine="python")
    filenames  = df["Id"].copy().values
    return filenames


def get_image(path: str, name: str, size: int) -> np.ndarray:
    image = cv2.imread(os.path.join(path, name + ".jpg"), cv2.IMREAD_COLOR)
    image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2RGB)
    image = cv2.resize(src=image, dsize=(size, size), interpolation=cv2.INTER_AREA)
    return image


def make_submission(path: str, y_pred: np.ndarray) -> None:
    submission = pd.read_csv(os.path.join(path, "sample_submission.csv"), engine="python")
    submission["Pawpularity"] = y_pred
    submission.to_csv("./submission.csv", index=False)

## Dataset Templates

In [5]:
class ImageDS(Dataset):
    def __init__(self, base_path=None, filenames=None, image_size=None, transform=None):
        self.base_path = base_path
        self.filenames = filenames
        self.image_size = image_size
        self.transform = transform
    
    def __len__(self):
        return self.filenames.shape[0]
    
    def __getitem__(self, idx):
        image = get_image(self.base_path, self.filenames[idx], self.image_size)
        return self.transform(image)

    
class FeatureDS(Dataset):
    def __init__(self, features=None):
        self.features = features
    
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.features[idx])

## Build Models

In [6]:
def build_models(IL: int, pretrained: bool, seed: int):

    class ImageModel(nn.Module):
        def __init__(self, pretrained=False):
            super(ImageModel, self).__init__()

            self.features = models.densenet121(pretrained=pretrained, progress=True)
            if pretrained:
                self.freeze()
            self.features = nn.Sequential(*[*self.features.children()][:-1])
            self.features.add_module("Adaptive Average Pool", nn.AdaptiveAvgPool2d(output_size=(1, 1)))
            self.features.add_module("Flatten", nn.Flatten())

        def freeze(self):
            for params in self.parameters():
                    params.requires_grad = False

        def forward(self, x):
            return self.features(x)
    
    torch.manual_seed(seed)
    vision_model = ImageModel(pretrained=pretrained)
    
    
    class ANN(nn.Module):
        def __init__(self, IL=None):
            super(ANN, self).__init__()

            self.predictor = nn.Sequential()
            self.predictor.add_module("BN", nn.BatchNorm1d(num_features=IL, eps=1e-5))
            self.predictor.add_module("FC", WN(nn.Linear(in_features=IL, out_features=1)))

        def get_optimizer(self, lr=1e-3, wd=0):
            params = [p for p in self.parameters() if p.requires_grad]
            return optim.Adam(params, lr=lr, weight_decay=wd)

        def get_plateau_scheduler(self, optimizer=None, patience=5, eps=1e-8):
            return optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=patience, eps=eps, verbose=True)

        def forward(self, x):
            return self.predictor(x)
    
    torch.manual_seed(seed)
    ann_model = ANN(IL=IL)
    
    return vision_model, ann_model

## Feature Getter and Predict Helper

In [7]:
def get_features(model=None, dataloader=None, feature_length=None, path=None):
    model.load_state_dict(torch.load(path, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()

    features = torch.zeros(1, feature_length).to(DEVICE)
    for X in dataloader:
        X = X.to(DEVICE)
        with torch.no_grad():
            output = model(X)
        features = torch.cat((features, output.view(-1, feature_length)), dim=0)
    
    return features[1:].detach().cpu().numpy()


def predict_batch(model=None, dataloader=None, mode="test", path=None):
    model.load_state_dict(torch.load(path, map_location=DEVICE)["model_state_dict"])
    model.to(DEVICE)
    model.eval()

    y_pred = torch.zeros(1, 1).to(DEVICE)
    if re.match(r"valid", mode, re.IGNORECASE):
        for X, _ in dataloader:
            X = X.to(DEVICE)
            with torch.no_grad():
                output = model(X)
            y_pred = torch.cat((y_pred, output.view(-1, 1)), dim=0)
    elif re.match(r"test", mode, re.IGNORECASE):
        for X in dataloader:
            X = X.to(DEVICE)
            with torch.no_grad():
                output = model(X)
            y_pred = torch.cat((y_pred, output.view(-1, 1)), dim=0)
    
    return y_pred[1:].detach().cpu().numpy()

## Generate Submission

In [8]:
def submit():
    ts_filenames = get_filenames(PATH)
    filenames, targets = get_filenames_and_targets(PATH)
    
    ts_image_data_setup = ImageDS(base_path=os.path.join(PATH, "test"), 
                                  filenames=ts_filenames, 
                                  image_size=IMAGE_SIZE, 
                                  transform=TRANSFORM)
    ts_image_data = DL(ts_image_data_setup, batch_size=64, shuffle=False)
    
    VisionModel, ANNModel = build_models(IL=FEATURE_LENGTH, pretrained=False, seed=SEED)
    
    ts_features = get_features(model=VisionModel, dataloader=ts_image_data, 
                               feature_length=FEATURE_LENGTH, 
                               path=os.path.join(PRETRAINED_WEIGHTS_PATH, "{}_state.pt".format(MODEL_NAME)))
    
    
    breaker()
    print("Making Predictions on Test Features ...")
    breaker()
    
    fold = 1
    final_y_pred = np.zeros((len(ts_filenames), 1))
    for tr_idx, va_idx in KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=SEED).split(filenames):
        print("Processing Fold {} ...".format(fold))
        
        tr_targets = targets[tr_idx]
        tr_targets = tr_targets.reshape(-1, 1)
        tr_targets = sc_y.fit_transform(tr_targets)
    
        ts_feature_data_setup = FeatureDS(features=ts_features)
        ts_feature_data = DL(ts_feature_data_setup, batch_size=512, shuffle=False)

        y_pred = predict_batch(model=ANNModel, dataloader=ts_feature_data, mode="test",
                               path=os.path.join(TRAINED_ANN_WEIGHTS_PATH, "Fold_{}_state.pt".format(fold)))
        y_pred = sc_y.inverse_transform(y_pred)
        
        final_y_pred += y_pred
        fold += 1
    
    final_y_pred = final_y_pred / NUM_FOLDS
    
    
    breaker()
    print("Generating Submission File ...")
    make_submission(PATH, final_y_pred)
    breaker()

submit()

RuntimeError: Error(s) in loading state_dict for ImageModel:
	Unexpected key(s) in state_dict: "features.0.denseblock3.denselayer25.norm1.weight", "features.0.denseblock3.denselayer25.norm1.bias", "features.0.denseblock3.denselayer25.norm1.running_mean", "features.0.denseblock3.denselayer25.norm1.running_var", "features.0.denseblock3.denselayer25.norm1.num_batches_tracked", "features.0.denseblock3.denselayer25.conv1.weight", "features.0.denseblock3.denselayer25.norm2.weight", "features.0.denseblock3.denselayer25.norm2.bias", "features.0.denseblock3.denselayer25.norm2.running_mean", "features.0.denseblock3.denselayer25.norm2.running_var", "features.0.denseblock3.denselayer25.norm2.num_batches_tracked", "features.0.denseblock3.denselayer25.conv2.weight", "features.0.denseblock3.denselayer26.norm1.weight", "features.0.denseblock3.denselayer26.norm1.bias", "features.0.denseblock3.denselayer26.norm1.running_mean", "features.0.denseblock3.denselayer26.norm1.running_var", "features.0.denseblo

In [ ]:
breaker()
print("Notebook Run Time : {:.2f} minutes".format((time()-notebook_start_time)/60))
breaker()